In [ ]:
pip install spacy vaderSentiment rake-nltk


In [ ]:
pip install spacy

In [73]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from rake_nltk import Rake

In [74]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4hara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\4hara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\4hara\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\4hara\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\4hara\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\4hara\AppData\Roaming\nltk_data...
[nltk_data]   Package words is

True

In [75]:
def summarize_text(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Calculate frequency of each word
    word_frequencies = {}
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        for word in words:
            if word not in stopwords.words('english'):
                if word not in word_frequencies:
                    word_frequencies[word] = 1
                else:
                    word_frequencies[word] += 1

    # Find weighted frequency of each word
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / max_frequency)

    # Calculate sentence scores based on word frequencies
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies.keys():
                if len(sentence.split(' ')) < 30:  # Limiting sentence length
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_frequencies[word]
                    else:
                        sentence_scores[sentence] += word_frequencies[word]

    # Get the top 3 sentences as the summary
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:3]
    summary = ' '.join(summary_sentences)
    return summary

In [76]:
def analyze_bias_and_sentiment(summary):
    # Sentiment Analysis
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(summary)
    sentiment = "Positive" if sentiment_score['compound'] >= 0 else "Negative"

    return sentiment, sentiment_score

In [77]:
def extract_article_content(url, token):
    # Construct the API request URL
    api_url = f"https://api.diffbot.com/v3/article"
    params = {
        'token': token,
        'url': url,
    }

    # Send the API request
    response = requests.get(api_url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if article extraction was successful
        if data.get('objects'):
            article_content = data['objects'][0]['text']
            return article_content
        else:
            return "Failed to extract article content."
    else:
        return "Failed to retrieve data from Diffbot."

# Example URL and API token
url = input("Paste your URL here.")
token = "72396479255cf4be659af8c87bd3d3f0"  # Replace this with your Diffbot API token

# Extract article contenthttps://housker.github.io/confidentscientist/es/posts/24-05-15/
article_content = extract_article_content(url, token)
print(article_content)

Paste your URL here. https://uk.news.yahoo.com/justice-dept-makes-arrests-north-201310276.html


WASHINGTON (AP) — The Justice Department announced Thursday multiple arrests in a series of complex stolen identity theft cases that officials say are part of a wide-ranging scheme that generates enormous proceeds for the North Korean government, including for its weapons program.
The conspiracy involves thousands of North Korean information technology workers who prosecutors say are dispatched by the government to live abroad and who rely on the stolen identities of Americas to obtain remote employment at U.S.-based Fortune 500 companies, jobs that give them access to sensitive corporate data and lucrative paychecks. The companies did not realize the workers were overseas.
The fraud scheme is a way for heavily sanctioned North Korea, which is cut off from the U.S. financial system, to take advantage of a “toxic brew” of converging factors, including a high-tech labor shortage in the U.S. and the proliferation of remote telework, Marshall Miller, the Justice Department's principal asso

In [78]:
def get_article_title(url, token):
# Construct the API request URL
    api_url = f"https://api.diffbot.com/v3/article"
    params = {
        'token': token,
        'url': url,
    }

    # Send the API request
    response = requests.get(api_url, params=params)

    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the title from the response
        if 'objects' in data and len(data['objects']) > 0:
            title = data['objects'][0].get('title', 'No title found')
            return title
        else:
            return 'No article objects found in the response'
    else:
        return f'Error: {response.status_code}'


title = get_article_title(url, token)
print("Article Title:", title)

Article Title: Justice Dept. makes arrests in North Korean identity theft scheme involving thousands of IT workers


In [79]:
#Text Input
input_text = article_content

# Summarize the input text
summary = summarize_text(input_text)

# Analyze bias and sentiment of the summary
sentiment, sentiment_score = analyze_bias_and_sentiment(summary)

print("Summary:")
print(summary)
print("\nSentiment:", sentiment)
print("Sentiment Score:", sentiment_score)

Summary:
“More and more often, compliance programs at American companies and organizations are on the front lines of protecting our national security,” Miller said. According to the indictment, Chapman ran more than one “laptop farm” where U.S. companies sent computers and paychecks to IT workers they did not realize were overseas. Separately, the State Department said it was offering a reward for information about certain North Korean IT workers who officials say were assisted by Chapman.

Sentiment: Positive
Sentiment Score: {'neg': 0.0, 'neu': 0.926, 'pos': 0.074, 'compound': 0.7003}


In [80]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from heapq import nlargest

def analyze_topic_and_summarize(text):
    # Tokenize the text into sentences and words
    sentences = sent_tokenize(text)

    # Tokenize words and filter out stop words
    stop_words = set(stopwords.words('english'))
    word_tokens = [word.lower() for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words]

    # Convert sentences to lower case
    sentences = [sentence.lower() for sentence in sentences]

    # Calculate TF-IDF scores for words
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(sentences)
    word_scores = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

    # Get the top keywords based on TF-IDF scores
    top_keywords = nlargest(5, word_scores, key=word_scores.get)

    # Generate summary sentences containing the top keywords
    summary_sentences = []
    for sentence in sentences:
        if any(keyword in sentence for keyword in top_keywords):
            summary_sentences.append(sentence)

    # Combine summary sentences into a single summary
    summary = ' '.join(summary_sentences)

    return top_keywords, summary

# Example input text
input_text = article_content

# Analyze the topic and summarize the input text
top_keywords, summary = analyze_topic_and_summarize(input_text)
print("Topic Keywords:", top_keywords)
print("\nSummary:")
print(summary)

Topic Keywords: ['300', '500', '____', 'access', 'according']

Summary:
the conspiracy involves thousands of north korean information technology workers who prosecutors say are dispatched by the government to live abroad and who rely on the stolen identities of americas to obtain remote employment at u.s.-based fortune 500 companies, jobs that give them access to sensitive corporate data and lucrative paychecks. "corporate compliance and national security are now intertwined like never before.”
the justice department said in court documents in one case that more than 300 companies — including a high-end retail chain and a “premier silicon valley technology company” — have been affected and that more than $6.8 million in revenue has been generated for the workers, who are based outside of the u.s., including in china and russia. according to the indictment, chapman ran more than one “laptop farm” where u.s. companies sent computers and paychecks to it workers they did not realize were o

In [82]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

def analyze_topic_and_summarize(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Initialize an empty list to store relevant entities
    relevant_entities = []

    # Perform part-of-speech tagging and named entity recognition for each sentence
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged_words = nltk.pos_tag(words)
        named_entities = nltk.ne_chunk(tagged_words)

        # Extract person entities and other relevant entities (organizations, geopolitical entities)
        for entity in named_entities:
            if isinstance(entity, nltk.tree.Tree):
                if entity.label() in ["PERSON", "ORGANIZATION", "GPE"]:
                    relevant_entities.extend([word for word, _ in entity.leaves()])

    # Get the top keywords
    top_keywords = set(relevant_entities[:5])  # Adjust as needed

    # Generate summary sentences containing the top keywords
    summary_sentences = []
    for sentence in sentences:
        if any(keyword.lower() in sentence.lower() for keyword in top_keywords):
            summary_sentences.append(sentence)

    # Combine summary sentences into a single summary
    summary = ' '.join(summary_sentences)

    return top_keywords, summary


# Analyze the topic and summarize the input text
summary = analyze_topic_and_summarize(article_content)
# print("Topic Keywords:", top_keywords)
print("\nSummary:")
print(summary)
print("\nSentiment:", sentiment)
print("Sentiment Score:", sentiment_score)


Summary:
({'AP', 'WASHINGTON', 'Department', 'Korean', 'Justice'}, 'WASHINGTON (AP) — The Justice Department announced Thursday multiple arrests in a series of complex stolen identity theft cases that officials say are part of a wide-ranging scheme that generates enormous proceeds for the North Korean government, including for its weapons program. The conspiracy involves thousands of North Korean information technology workers who prosecutors say are dispatched by the government to live abroad and who rely on the stolen identities of Americas to obtain remote employment at U.S.-based Fortune 500 companies, jobs that give them access to sensitive corporate data and lucrative paychecks. The fraud scheme is a way for heavily sanctioned North Korea, which is cut off from the U.S. financial system, to take advantage of a “toxic brew” of converging factors, including a high-tech labor shortage in the U.S. and the proliferation of remote telework, Marshall Miller, the Justice Department\'s p

In [81]:
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the English language model for SpaCy
nlp = spacy.load("en_core_web_sm")

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    # Perform sentiment analysis using VADER
    sentiment_scores = sid.polarity_scores(text)
    compound_score = sentiment_scores['compound']

    # Determine sentiment label based on compound score
    sentiment = "Positive" if compound_score >= 0.05 else "Negative" if compound_score <= -0.05 else "Neutral"
    return sentiment

def analyze_for_or_against_idea(text, idea):
    # Process the text using SpaCy
    doc = nlp(text)

    # Extract entities and analyze sentiment towards them
    entities_sentiment = {}
    for ent in doc.ents:
        if ent.text.lower() == idea.lower():
            continue  # Skip the idea itself
        entity_sentiment = analyze_sentiment(ent.text)
        entities_sentiment[ent.text] = entity_sentiment

    # Determine overall sentiment towards the idea
    idea_sentiment = analyze_sentiment(idea)

    # Determine if the overall sentiment is for or against the idea
    if idea_sentiment == "Positive":
        sentiment_label = "Supportive"
    elif idea_sentiment == "Negative":
        sentiment_label = "Critical"
    else:
        sentiment_label = "Neutral"

    return sentiment_label, entities_sentiment

# Example idea
idea = title

# Analyze sentiment towards the idea and entities mentioned in the text
sentiment_label, entities_sentiment = analyze_for_or_against_idea(article_content, idea)

# Print results
print("Overall Sentiment Label towards the Idea:", sentiment_label)
print("Sentiment towards Entities:")
print("\nSummary:")
print(summary)

Overall Sentiment Label towards the Idea: Supportive
Sentiment towards Entities:

Summary:
the conspiracy involves thousands of north korean information technology workers who prosecutors say are dispatched by the government to live abroad and who rely on the stolen identities of americas to obtain remote employment at u.s.-based fortune 500 companies, jobs that give them access to sensitive corporate data and lucrative paychecks. "corporate compliance and national security are now intertwined like never before.”
the justice department said in court documents in one case that more than 300 companies — including a high-end retail chain and a “premier silicon valley technology company” — have been affected and that more than $6.8 million in revenue has been generated for the workers, who are based outside of the u.s., including in china and russia. according to the indictment, chapman ran more than one “laptop farm” where u.s. companies sent computers and paychecks to it workers they did